# Hoja de trabajo 8 Máquinas Vectoriales de Soporte

repo: https://github.com/Diegoval-Dev/DM-HDT8
- Gerson Ramirez - 22281
- Diego Valenzuela - 22309

In [1]:

import pandas as pd
import numpy as np

df = pd.read_csv("processed_data.csv")

print(f"Dimensiones del dataset: {df.shape}")
df.head()


Dimensiones del dataset: (1460, 77)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PriceCategory
0,1,60,3,65.0,8450,1,3,3,0,4,...,0,0,0,0,2,2008,8,4,208500,Cara
1,2,20,3,80.0,9600,1,3,3,0,2,...,0,0,0,0,5,2007,8,4,181500,Intermedia
2,3,60,3,68.0,11250,1,0,3,0,4,...,0,0,0,0,9,2008,8,4,223500,Cara
3,4,70,3,60.0,9550,1,0,3,0,0,...,0,0,0,0,2,2006,8,0,140000,Intermedia
4,5,60,3,84.0,14260,1,0,3,0,2,...,0,0,0,0,12,2008,8,4,250000,Cara


## Separación de conjuntos de entrenamiento y prueba

In [4]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["SalePrice", "PriceCategory"])
y = df["PriceCategory"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()


((1168, 75),
 (292, 75),
 Intermedia    392
 Economica     390
 Cara          386
 Name: PriceCategory, dtype: int64,
 Intermedia    98
 Cara          97
 Economica     97
 Name: PriceCategory, dtype: int64)

## Exploración de variables y transformaciones para SVM

Las máquinas vectoriales de soporte (SVM) son algoritmos sensibles a la escala de las variables, por lo que es necesario estandarizar los datos antes de entrenar los modelos. Además, las variables categóricas deben ser transformadas a formato numérico mediante codificación one-hot.

En este paso, se inspeccionarán las características de `X_train` para determinar:
- Qué variables son numéricas y cuáles categóricas.
- Si existen valores nulos que deben ser imputados.
- Las transformaciones necesarias para asegurar compatibilidad con los modelos de SVM.

Posteriormente, se construirá un pipeline de preprocesamiento que integre imputación, codificación y escalamiento, utilizando `ColumnTransformer` y `Pipeline` de `sklearn`.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape, X_test_processed.shape


((1168, 75), (292, 75))

## Análisis de las transformaciones aplicadas

Tras aplicar el pipeline de preprocesamiento, el conjunto de entrenamiento (`X_train`) pasó de su representación original a una matriz de características con 75 columnas. Este aumento en dimensionalidad se debe a la codificación one-hot aplicada a las variables categóricas, que crea una columna binaria por cada categoría única.

Además, las variables numéricas fueron estandarizadas con media cero y desviación estándar uno, lo que es esencial para garantizar que las SVM no se vean dominadas por características con escalas mayores.

Esta transformación asegura que el modelo pueda trabajar en un espacio vectorial consistente, donde cada dimensión tiene un impacto comparable en el margen de separación. Así, se cumplen los requerimientos técnicos previos al entrenamiento de máquinas de soporte vectorial.

## Entrenamiento de modelos SVM con diferentes kernels

Para evaluar el comportamiento de los clasificadores SVM bajo distintas configuraciones, se entrenarán tres modelos utilizando los siguientes kernels:

- **Lineal**: intenta encontrar un hiperplano lineal que separe las clases.
- **Radial (RBF)**: proyecta los datos a un espacio no lineal, permitiendo separar clases que no son linealmente separables.
- **Polinomial**: permite aprender relaciones de orden superior entre las variables predictoras.

Todos los modelos se entrenarán sobre el conjunto de entrenamiento ya preprocesado, y se evaluarán posteriormente sobre el conjunto de prueba para analizar su capacidad generalizadora.


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import time

results = {}

# Modelo SVM lineal
start = time.time()
svm_linear = SVC(kernel="linear", C=1)
svm_linear.fit(X_train_processed, y_train)
end = time.time()
y_pred_linear = svm_linear.predict(X_test_processed)
results["linear"] = {
    "y_pred": y_pred_linear,
    "fit_time": end - start,
    "conf_matrix": confusion_matrix(y_test, y_pred_linear),
    "report": classification_report(y_test, y_pred_linear, output_dict=True)
}

# Modelo SVM RBF
start = time.time()
svm_rbf = SVC(kernel="rbf", C=1, gamma="scale")
svm_rbf.fit(X_train_processed, y_train)
end = time.time()
y_pred_rbf = svm_rbf.predict(X_test_processed)
results["rbf"] = {
    "y_pred": y_pred_rbf,
    "fit_time": end - start,
    "conf_matrix": confusion_matrix(y_test, y_pred_rbf),
    "report": classification_report(y_test, y_pred_rbf, output_dict=True)
}

# Modelo SVM polinomial
start = time.time()
svm_poly = SVC(kernel="poly", C=1, degree=3, gamma="scale")
svm_poly.fit(X_train_processed, y_train)
end = time.time()
y_pred_poly = svm_poly.predict(X_test_processed)
results["poly"] = {
    "y_pred": y_pred_poly,
    "fit_time": end - start,
    "conf_matrix": confusion_matrix(y_test, y_pred_poly),
    "report": classification_report(y_test, y_pred_poly, output_dict=True)
}

results


{'linear': {'y_pred': array(['Intermedia', 'Cara', 'Economica', 'Intermedia', 'Intermedia',
         'Economica', 'Intermedia', 'Economica', 'Cara', 'Cara',
         'Intermedia', 'Intermedia', 'Economica', 'Economica', 'Intermedia',
         'Economica', 'Economica', 'Intermedia', 'Economica', 'Economica',
         'Cara', 'Cara', 'Cara', 'Cara', 'Cara', 'Cara', 'Cara', 'Cara',
         'Cara', 'Economica', 'Intermedia', 'Intermedia', 'Cara',
         'Intermedia', 'Cara', 'Economica', 'Cara', 'Cara', 'Intermedia',
         'Intermedia', 'Cara', 'Intermedia', 'Cara', 'Cara', 'Economica',
         'Intermedia', 'Economica', 'Cara', 'Cara', 'Cara', 'Economica',
         'Economica', 'Economica', 'Economica', 'Cara', 'Cara', 'Cara',
         'Cara', 'Economica', 'Cara', 'Economica', 'Economica',
         'Intermedia', 'Cara', 'Intermedia', 'Economica', 'Economica',
         'Cara', 'Cara', 'Economica', 'Cara', 'Economica', 'Cara', 'Cara',
         'Economica', 'Intermedia', 'Cara', 'Inte

## Análisis comparativo de modelos SVM (kernels lineal, radial y polinomial)

Se entrenaron tres modelos SVM con diferentes kernels utilizando los mismos datos transformados. A continuación, se presentan los principales hallazgos con base en las métricas de evaluación y tiempos de entrenamiento.

### Kernel lineal
- **Accuracy**: 0.7945
- **Precisión promedio**: 0.7926
- **F1-score promedio**: 0.7930
- **Tiempo de entrenamiento**: 0.31 s
- Buen desempeño general, especialmente en las clases "Cara" y "Económica".
- La clase "Intermedia" mostró menor recall (0.67), lo que indica dificultad para capturar correctamente los ejemplos de esta clase.

### Kernel radial (RBF)
- **Accuracy**: 0.8151
- **Precisión promedio**: 0.8125
- **F1-score promedio**: 0.8126
- **Tiempo de entrenamiento**: 0.087 s
- Supera al kernel lineal tanto en precisión como en F1-score.
- El tiempo de entrenamiento fue significativamente menor.
- Las tres clases muestran un buen equilibrio en precisión y recall, con "Intermedia" siendo nuevamente la más difícil de clasificar.

### Kernel polinomial (grado 3)
- **Accuracy**: 0.7192
- **Precisión promedio**: 0.7838
- **F1-score promedio**: 0.7263
- **Tiempo de entrenamiento**: 0.12 s
- Peor desempeño general, especialmente en la clase "Económica", con un recall de solo 0.60.
- A pesar de alta precisión en "Cara" y "Económica", el modelo comete muchos falsos negativos.

### Conclusiones
El modelo con kernel **radial (RBF)** presentó el mejor rendimiento general. Fue el único en superar el 81% de exactitud, manteniendo buen balance entre clases y ejecutándose en el menor tiempo. Esto lo convierte en el candidato más sólido para continuar con procesos de validación cruzada y ajuste de hiperparámetros.

El modelo polinomial no logró una separación eficiente de clases, lo cual podría explicarse por un grado insuficiente o sobreajuste a patrones complejos sin generalización adecuada.
